In [3]:
# OP
import pandas as pd 
import os 
# spark 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

# spark config 
conf = SparkConf().setAppName("LOAD PTT MYSQL DATABASE")
#sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)


def get_mysql_creds():
	url = "jdbc:mysql://{}/{}".format(HOST, DATABASE)
	creds = {"url":url,
	"connectionProperties" : {
	"user" : USER,
	"password" : PASSWORD}}
	return creds 


def get_ptt_table_data(creds, table):
	creds = get_mysql_creds()
	spark_df = sqlContext.read.jdbc(url=creds['url'],properties=creds['connectionProperties'], table=table )
	pandas_df = spark_df.toPandas()
	return spark_df, pandas_df 


def query_spark_SQL(spark_df):
	spark_df.registerTempTable("temp_sql_table")
	spark_sql_output=sqlContext.sql(""" SELECT 
	DATE(date),
	author_ip,
	count(*)
	from temp_sql_table
	group by 1 
	order by 1 desc""") 
	print (spark_sql_output.take(10))
	return spark_sql_output


In [4]:
HOST = '103.29.68.107'
USER = 'guest'
PASSWORD = '123'
DATABASE = 'PTTData'



In [5]:
creds = get_mysql_creds()
spark_df, pandas_df  = get_ptt_table_data(creds, "Soft_Job")
#spark_df, pandas_df  = get_ptt_table_data(creds, "Gossiping")
print ('='*70)
print ('spark_df : ', spark_df.collect())
print (type(spark_df))
print ('pandas_df : ', pandas_df)
print (type(pandas_df))

Py4JJavaError: An error occurred while calling o27.jdbc.
: java.sql.SQLException: No suitable driver
	at java.sql.DriverManager.getDriver(DriverManager.java:315)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$7.apply(JDBCOptions.scala:85)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$7.apply(JDBCOptions.scala:85)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:84)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:35)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:34)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:340)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:239)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:227)
	at org.apache.spark.sql.DataFra